In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import seaborn as sns
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score
import matplotlib.pyplot as plt
import nltk
import textblob 

In [26]:
yelp_train_businesses = pd.read_csv("yelp_csv_dataset/yelp_train_businesses.csv")
yelp_train_tips = pd.read_csv("yelp_csv_dataset/yelp_train_tips.csv")
yelp_train_users = pd.read_csv("yelp_csv_dataset/yelp_train_users.csv")
yelp_train_checkins = pd.read_csv("yelp_csv_dataset/yelp_train_checkins.csv")
yelp_train_reviews = pd.read_csv("yelp_csv_dataset/yelp_train_reviews.csv")

yelp_test_businesses = pd.read_csv('yelp_csv_dataset/yelp_test_businesses.csv')
yelp_validate_businesses = pd.read_csv('yelp_csv_dataset/yelp_validate_businesses.csv')

In [27]:
train_df = yelp_train_businesses[['postal_code', 'stars', 'attributes', 'categories', 'hours']]
train_df = train_df[train_df['categories'].str.contains('restaurant', case=False, na=False)]
train_df.dropna(inplace=True)

train_df_sample = train_df.sample(n=10000, random_state=42)

train_x = train_df_sample[['postal_code', 'attributes', 'categories', 'hours']]
train_y = train_df_sample['stars']

In [28]:
validate_df = yelp_validate_businesses[['postal_code', 'stars', 'attributes', 'categories', 'hours']]
validate_df = validate_df[validate_df['categories'].str.contains('restaurant', case=False, na=False)]
validate_df.dropna(inplace=True)

validate_df_sample = validate_df.sample(n=10000, random_state=42)

val_x = validate_df_sample[['postal_code', 'attributes', 'categories', 'hours']]
val_y = validate_df_sample['stars']

In [29]:
test_df = yelp_test_businesses[['postal_code', 'stars', 'attributes', 'categories', 'hours']]
test_df = test_df[test_df['categories'].str.contains('restaurant', case=False, na=False)]
test_df.dropna(inplace=True)


test_x = test_df[['postal_code', 'attributes', 'categories', 'hours']]
test_y = test_df['stars']

In [30]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from scipy.sparse import hstack, csr_matrix

encoder_x = OneHotEncoder(handle_unknown='ignore', sparse_output=True)
encoder_x.fit(train_x)
x_train_sparse = encoder_x.transform(train_x)
x_val_sparse = encoder_x.transform(val_x)
x_test_sparse = encoder_x.transform(test_x)

In [31]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(x_train_sparse, train_y)

LinearRegression()

In [10]:

val_accuracy = lr.score(x_val_sparse, val_y)
print("Validation Accuracy:", val_accuracy)

Validation Accuracy: -1.8721115182443162


In [32]:
from sklearn.metrics import mean_squared_error
predictions = lr.predict(x_test_sparse)

mse = mean_squared_error(test_y, predictions)
print("Mean Squared Error:", mse)

Mean Squared Error: 0.522368973985817
